In [51]:
train_projects = [
    # "2023-08-28_FW_EingangBauing.FwfProj",                      # building
    "2024-03-22_FW_Koenigshuegel.FwfProj",                      # building
    # "2024-04-05_FW_Westbahnhof_01.FwfProj",                     # tunnel-bridge
    "2024-04-05_FW_Westbahnhof_02.FwfProj",                     # tunnel-bridge
    "2024-04-05_FW_Westbahnhof_03.FwfProj",                     # tunnel-bridge
    "2024-04-05_FW_Westbahnhof_04.FwfProj",                     # tunnel-bridge
    "2024-04-05_FW_Westbahnhof_05.FwfProj",                     # building
    "2024-05-10_FW_RWTH_Zentrum_01.FwfProj",                    # building
    "2024-07-31_FW_Bruecke_Koenigstr.FwfProj",                  # bridge (steel+stone) --> split?
    # "2024-07-31_FW_Bruecke_Turmstr.FwfProj",                    # bridge (concrete/steel)
    "2024-08-02_FW_Bruecke_A44_VerlautenheidenerStr.FwfProj",   # bridge (concrete)
    "2024-08-02_FW_Bruecke_Deltourserb.FwfProj",                # bridge (concrete)
    "2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj",              # bridge (concrete, steel, brick)
    # "2024-08-02_FW_Bruecke_RotheErde.FwfProj",                  # bridge (steel)
    "2024-08-02_FW_Bruecke_Rottstrasse.FwfProj",                # bridge (concrete)
]
val_projects = [
    "2023-08-28_FW_EingangBauing.FwfProj",                      # building
    "2024-04-05_FW_Westbahnhof_01.FwfProj",                     # tunnel-bridge
    "2024-07-31_FW_Bruecke_Turmstr.FwfProj",                    # bridge (concrete/steel)
    "2024-08-02_FW_Bruecke_RotheErde.FwfProj",                  # bridge (steel)
]

In [62]:
from plyfile import PlyData, PlyElement
import pandas as pd
import numpy as np
from glob import glob
import json
import os
from omegaconf import OmegaConf
from scipy.spatial import KDTree

data_dir = "../../02_Datasets/FWF_Aachen_labeled/*.FwfProj"




proj_names_list = train_projects

projects = list()
for fwf_prof_fp in glob(data_dir):
    proj_name = os.path.basename(fwf_prof_fp)
    if proj_name not in proj_names_list:
        continue
    
    print(proj_name)

    # load the data
    pcd = pd.DataFrame(PlyData.read(list(glob(os.path.join(fwf_prof_fp,'labeled','*pointcloud.ply')))[0]).elements[0].data)
    wfm = np.load(list(glob(os.path.join(fwf_prof_fp,'labeled','*waveform.npy')))[0])
    spo = json.load(open(list(glob(os.path.join(fwf_prof_fp,'labeled','*metadata.json')))[0],"r"))

    # pre-calculations
    xyz = pcd[['x','y','z']].to_numpy()
    kd_tree = KDTree(data=xyz)

    # get scan positions
    if 'scanId=000' in spo['scan_positions'].keys():
        spo_sop = np.array([spo['scan_positions'][f'scanId={si:03}']['sop'] for si in range(len(spo['scan_positions']))])
    else:
        # handle case where only one scan position is in the metadata and it's f.s.r. labeled 'scanId=001' instead of 'scanID=000'
        spo_sop = np.array([spo['scan_positions']['scanId=001']['sop']])



    # field names
    riegl_feat_names = ['riegl_reflectance',
            'riegl_amplitude', 'riegl_deviation', 'riegl_targetIndex',
            'riegl_targetCount']
    label_names = ['labels_0', 'labels_1', 'labels_2', 'labels_3']


    # save project as dict
    projects.append(dict(
        proj_name=proj_name,
        wfm = wfm,
        spo_sop = spo_sop,
        rgb = pcd[['Red','Green','Blue']].to_numpy(),
        riegl_feats = pcd[riegl_feat_names].to_numpy(),
        labels = pcd[label_names].to_numpy(),
        spo_ids = pcd['scan_id'].to_numpy()
    ))

    # if proj_name == "2024-07-31_FW_Bruecke_Koenigstr.FwfProj": break
    print(projects[-1]['spo_sop'].shape)
    

    


    



2024-03-22_FW_Koenigshuegel.FwfProj
(1, 4, 4)
2024-04-05_FW_Westbahnhof_02.FwfProj
(1, 4, 4)
2024-04-05_FW_Westbahnhof_03.FwfProj
(1, 4, 4)
2024-04-05_FW_Westbahnhof_04.FwfProj
(1, 4, 4)
2024-04-05_FW_Westbahnhof_05.FwfProj
(1, 4, 4)
2024-05-10_FW_RWTH_Zentrum_01.FwfProj
(1, 4, 4)
2024-07-31_FW_Bruecke_Koenigstr.FwfProj
(9, 4, 4)
2024-08-02_FW_Bruecke_A44_VerlautenheidenerStr.FwfProj
(5, 4, 4)
2024-08-02_FW_Bruecke_Deltourserb.FwfProj
(2, 4, 4)
2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj
(7, 4, 4)
2024-08-02_FW_Bruecke_Rottstrasse.FwfProj
(3, 4, 4)


In [55]:
spo

{'scan_contents': 'bridge_scan',
 'scan_positions': {'scanId=000': {'scan_pos_name': 'P001_240731_122741',
   'sop': [[0.9999970849471599,
     1.713553157023902e-05,
     0.0024144986139782924,
     0.0],
    [0.0, 0.9999748177320125, -0.00709675290741126, 0.0],
    [-0.0024145594180606305, 0.00709673222000154, 0.9999719027525794, 0.0],
    [0.0, 0.0, 0.0, 1.0]]},
  'scanId=001': {'scan_pos_name': 'P002_240731_123049',
   'sop': [[0.9998789083304942,
     0.006558502774758895,
     0.014112218719009044,
     -0.9389311308315091],
    [-0.007047793355074082,
     0.9993657593578653,
     0.034905696267254455,
     -9.83929346514609],
    [-0.013874339070523071,
     -0.035000929479531366,
     0.9992909674618926,
     -0.5177226175221026],
    [0.0, 0.0, 0.0, 1.0]]},
  'scanId=002': {'scan_pos_name': 'P003_240731_123347',
   'sop': [[0.9945294701785976,
     -0.09576400271984795,
     -0.04171796650546346,
     -11.573698866581935],
    [0.09770588859545336,
     0.9940906442120142,
  